In [ ]:
import psycopg2
import pandas as pd
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Connect to the database
conn = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),
    dbname=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD")
)

print("Step 1 ..")
# Step 1: Extract current drug formulary and tier info
query_formulary = """
SELECT formulary_id, rxcui, ndc, tier_level_value, quantity_limit_yn, prior_authorization_yn, step_therapy_yn
FROM basic_drugs_formulary
WHERE contract_year = (SELECT MAX(contract_year) FROM basic_drugs_formulary)
"""
formulary_df = pd.read_sql_query(query_formulary, conn)

print("Step 2 ..")
# Step 2: Extract beneficiary cost info grouped by rxcui and tier
query_costs = """
SELECT segment_id, rxcui, tier, 
       AVG(cost_amt_pref) AS avg_cost_pref,
       AVG(cost_amt_nonpref) AS avg_cost_nonpref,
       AVG(cost_amt_mail_pref) AS avg_cost_mail_pref,
       AVG(cost_amt_mail_nonpref) AS avg_cost_mail_nonpref
FROM beneficiary_cost
GROUP BY segment_id, rxcui, tier
"""
costs_df = pd.read_sql_query(query_costs, conn)

print("Step 3 ..")
# Step 3: Extract prescription volume and sales from prescribers_by_geography_drug table
query_prescriptions = """
SELECT rxcui, brnd_name, gnrc_name, 
       SUM(tot_30day_fills) AS total_30day_fills, 
       SUM(tot_drug_cst) AS total_drug_cost
FROM prescribers_by_geography_drug
GROUP BY rxcui, brnd_name, gnrc_name
"""
prescriptions_df = pd.read_sql_query(query_prescriptions, conn)

print("Step 4 ..")
# Step 4: Merge dataframes to analyze tier upgrade candidates
merged_df = (formulary_df
             .merge(costs_df, how='left', left_on=['rxcui', 'tier_level_value'], right_on=['rxcui', 'tier'])
             .merge(prescriptions_df, how='left', on='rxcui'))

print("Step 5 ..")
# Step 5: Identify drugs with high cost burden and potential for tier improvement
# Criteria: high average preferred cost at current tier, no prior auth or step therapy restrictions
candidates_df = merged_df[
    (merged_df['avg_cost_pref'] > merged_df['avg_cost_pref'].quantile(0.75)) &
    (merged_df['prior_authorization_yn'] == 'N') &
    (merged_df['step_therapy_yn'] == 'N')
]

print("Step 6 ..")
# Step 6: Estimate potential impact by simulating a tier downgrade (e.g. moving to one tier lower)
# Assume cost reduction proportional to tier change and estimate growth in 30day_fills (example heuristic)
candidates_df['potential_new_tier'] = candidates_df['tier_level_value'] - 1
candidates_df['estimated_cost_reduction'] = candidates_df['avg_cost_pref'] * 0.3  # assuming 30% cost reduction
candidates_df['estimated_fill_increase'] = candidates_df['total_30day_fills'] * 0.20  # assuming 20% fill increase

print("Step 7 ..")
# Summarize important columns
result_df = candidates_df[[
    'rxcui', 'ndc', 'tier_level_value', 'potential_new_tier', 
    'avg_cost_pref', 'estimated_cost_reduction', 
    'total_30day_fills', 'estimated_fill_increase',
    'total_drug_cost'
]].sort_values(by='estimated_fill_increase', ascending=False)

# Print or save results
print(result_df.head(20))

# Close connection
conn.close()
